<H3>PRI 2023/24: first project delivery</H3>

**GROUP 11**
- Francisco Martins, 99068
- Tunahan Güneş, 108108
- Sebastian Weidinger, 111612

<H3>Part I: demo of facilities</H3>

In [2]:
import os

In [14]:
def read_files(path):
    texts = []
    for folder in os.listdir(path):
        category_path = os.path.join(path, folder)
        texts.append([])
        for file in os.listdir(category_path):
            file_path = os.path.join(category_path, file)
            with open(file_path, "r", errors="ignore") as f:
                text = f.read()
                texts[-1].append(text)
                
    print("Number of Categories:",len(os.listdir(path)))
    for i in range(len(os.listdir(path))):
        print("Number of Articles in", "'"+os.listdir(path)[i]+"'", "Category:",len(texts[i]))
    return texts

In [55]:
dataset_path = os.path.join("BBC News Summary", "BBC News Summary", "News Articles")
print("Dataset path:", dataset_path)
categorized_articles = read_files(dataset_path)

Dataset path: BBC News Summary/BBC News Summary/News Articles
Number of Categories: 5
Number of Articles in 'tech' Category: 401
Number of Articles in 'entertainment' Category: 386
Number of Articles in 'sport' Category: 511
Number of Articles in 'business' Category: 510
Number of Articles in 'politics' Category: 417


In [57]:
#Examplary text. The structure of the read file is: articles[category_no][document_no]. 
print(categorized_articles[0][0])

Bond game fails to shake or stir

For gaming fans, the word GoldenEye evokes excited memories not only of the James Bond revival flick of 1995, but also the classic shoot-em-up that accompanied it and left N64 owners glued to their consoles for many an hour.

Adopting that hallowed title somewhat backfires on this new game, for it fails to deliver on the promise of its name and struggles to generate the original's massive sense of fun. This however is not a sequel, nor does it relate to the GoldenEye film. You are the eponymous renegade spy, an agent who deserts to the Bond world's extensive ranks of criminal masterminds, after being deemed too brutal for MI6. Your new commander-in-chief is the portly Auric Goldfinger, last seen in 1964, but happily running around bent on world domination. With a determination to justify its name which is even less convincing than that of Tina Turner's similarly-titled theme song, the game literally gives the player a golden eye following an injury, wh

A) **Indexing** (preprocessing and indexing options)

In [58]:
#code, statistics and/or charts here

imports

In [63]:
import time 
from typing import Union
import sklearn
import nltk

In [59]:
# flatten list to get uncategorized collection 
def flatten(catogorized_articles) -> list: 
    return [doc for category in catogorized_articles for doc in category]
articles = flatten(categorized_articles)

# Inverted Index Structure 
 
Each sentence points to terms: 

s1 -> t1 -> t2 -> t10\
s2 -> t3 -> t4 ...\
...

Save for each term in the document: 
* term frequency for each term in the document
* document frequency in the whole collection

In [67]:
'''
indexing(D,args)
    @input document collection D and optional arguments on text preprocessing
    @behavior preprocesses the collection and, using existing libraries, builds an inverted index with the relevant statistics for the subsequent summarization functions
    @output pair with the inverted index I and indexing time
'''

def indexing(articles, args=None) -> Union[dict, int]:
    inverted_index = dict()
    start_time = time.time()

    # loop through collection 
    for article in articles: 
        # split into sentences
        sents = nltk.sent_tokenize(article)
        # remove title (not needed for summarisation task)
        sents = sents[1:]
        for sent in sents: 
            words = nltk.word_tokenize(sent)
            t = 0


B) **Summarization**

*B.1 Summarization solution: results for a given document*

In [ ]:
#code, statistics and/or charts here

*B.2 IR models (TF-IDF, BM25 and EBRT)*

In [ ]:
#code, statistics and/or charts here

*B.3 Reciprocal rank funsion*

In [ ]:
#code, statistics and/or charts here

*B.4 Maximal Marginal Relevance*

In [ ]:
#code, statistics and/or charts here

C) **Keyword extraction**

In [ ]:
#code, statistics and/or charts here

D) **Evaluation**

In [ ]:
#code, statistics and/or charts here

<H3>Part II: questions materials (optional)</H3>

**(1)** Corpus *D* and summaries *S* description.

In [ ]:
#code, statistics and/or charts here

**(2)** Summarization performance for the overall and category-conditional corpora.

In [ ]:
#code, statistics and/or charts here

**...** (additional questions with empirical results)

<H3>END</H3>